In [112]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import requests
from selenium import webdriver
import time

In [113]:
import tqdm
import time
import json
import pandas as pd

In [114]:
#r = requests.get('https://aibee.com.tw/address/')
#r.encoding = "UTF-8"
#soup = BeautifulSoup(r.text, 'html.parser')
basic = 'https://aibee.com.tw'
basic_register = 'https://aibee.com.tw/br/'
basic_address = 'https://aibee.com.tw/address/'

In [115]:
def get_link_register(basic) :
    
    ## grasp page
    r = requests.get(basic+'/br/')
    r.encoding = "UTF-8"
    soup = BeautifulSoup(r.text, 'html.parser')
    
    ## get all url (company and commercial)
    
    ## get page list
    page_list = list(map(lambda x:x.get('href'),soup.find(class_ ='NavPg p0').find_all('a')))[:-2]  ## <-- cut first and last
    
    title_register_company = []
    href_register_company = []
    title_register_commercial = []
    href_register_commercial = []
    for page in page_list :
    
        r = requests.get(basic+page)
        r.encoding = "UTF-8"
        soup = BeautifulSoup(r.text, 'html.parser')     
    
        link_company = soup.find_all(class_ = 'p0')[0].find_all('a')
        title_register_company.extend(list(map(lambda x :x.get('title'),link_company)))
        href_register_company.extend(list(map(lambda x :basic + x.get('href'),link_company)))
        
        link_commercial = soup.find_all(class_ = 'p0')[1].find_all('a')
        title_register_commercial.extend(list(map(lambda x :x.get('title'),link_commercial)))
        href_register_commercial.extend(list(map(lambda x :basic + x.get('href'),link_commercial)))    
        
        
    register_company = dict(zip(title_register_company,href_register_company))
    register_commercial = dict(zip(title_register_commercial,href_register_commercial))
    return register_company , register_commercial

In [116]:
def get_link_address (basic) :
    
    r = requests.get(basic + '/address/')
    r.encoding = "UTF-8"
    soup = BeautifulSoup(r.text, 'html.parser')
    
    title_address_city = []
    href_address_city = []
    for city in range(len(soup.find(class_ = 'content').find_all('li'))) :
        
        link_address = soup.find(class_ = 'content').find_all('li')[city].find_all('a')

        title_address_city.extend(list(map(lambda x:x.get('title'),link_address)))
        href_address_city.extend(list(map(lambda x:basic + x.get('href'),link_address)))
        
    address_city_link = dict(zip(title_address_city,href_address_city))
    
    address_city_link = pd.DataFrame.from_dict(address_city_link,orient='index').reset_index()
    address_city_link.columns = ['address_city','address_url']
    
    return address_city_link

In [117]:
def get_link_address_sub (basic,address_city_link_title,address_city_link_url) :

    address_city_name = address_city_link_title
    r = requests.get(address_city_link_url)
    r.encoding = "UTF-8"
    soup = BeautifulSoup(r.text, 'html.parser')

    total_company = soup.find(class_ = 'content').find('h2').text
    
    basic_page = soup.find(class_ ='NavPg p0').find_all('a')[0].get('href')
    
    try :
        page = (int(soup.find(class_ ='NavPg p0').find_all('a')[-1].get('href')[-4:-1]) -1)  /20
    except :
        page = (int(soup.find(class_ ='NavPg p0').find_all('a')[-1].get('href')[-3:-1]) -1)  /20
    
    
    title_address_city_town = []
    href_address_city_town = []
    for i in range(0,int(page+1)) :
        if i == 0 :
            r = requests.get(basic + basic_page)
            r.encoding = "UTF-8"
            soup = BeautifulSoup(r.text, 'html.parser')
            
            title_address_city_town.extend(list(map(lambda x:x.find('a').get('title'),soup.find_all('li',class_='p0 BbHr'))))
            href_address_city_town.extend(list(map(lambda x:basic + x.find('a').get('href'),soup.find_all('li',class_='p0 BbHr'))))
            
        else :
            r = requests.get(basic + basic_page  +str(i*20 + 1)+'/')
            r.encoding = "UTF-8"
            soup = BeautifulSoup(r.text, 'html.parser')
            title_address_city_town.extend(list(map(lambda x:x.find('a').get('title'),soup.find_all('li',class_='p0 BbHr'))))
            href_address_city_town.extend(list(map(lambda x:basic + x.find('a').get('href'),soup.find_all('li',class_='p0 BbHr'))))
            
    
    city_company = dict(zip(title_address_city_town,href_address_city_town))
    
    city_company = pd.DataFrame.from_dict(city_company,orient='index').reset_index()
    city_company.columns = ['city_company','city_company_url']
    city_company['total_company'] = total_company
    city_company['address_city'] = address_city_name
    
    return city_company

In [72]:
def get_detail_address_sub (basic,address_city_name,city_company_title,city_company_url) :
    
    ### get 資本額等等 
    
    address_city = address_city_name
    city_company = city_company_title
    
    r = requests.get(city_company_url)  ### address_city_town is the url of the company !! 
                                         ### example : 'https://aibee.com.tw/v/26747432/'
    r.encoding = "UTF-8"
    soup = BeautifulSoup(r.text, 'html.parser')    
    
    key = []
    val = []
    count = 1
    for content in soup.find('div',class_='content').find_all('span') :

        if (count % 2 != 0 and count <=15):
            key.append(content.text)

        else :
            val.append(content.text)

        count += 1
    
    count_category = 0
    count_industry = 0
    for content in soup.find('div',class_='content').find_all('a') :

        if ('category' in content.get('href') and count_category <=10)  :
            key.append(f'category{count_category}')
            val.append(content.text)
            count_category +=1
        elif ('industry' in content.get('href') and count_industry <=10) :
            key.append(f'industry{count_industry}')
            val.append(content.text)
            count_industry +=1
        else :
            pass
        
    company_detail = dict(zip(key,val))    
    
    company_detail = pd.DataFrame.from_dict(company_detail,orient='index').T 
    company_detail['address_city'] = address_city
    company_detail['city_company'] = city_company
    
    return company_detail



### 轉成 List 即可直接轉為dataframe

In [124]:
# test
df_city_link = get_link_address(basic)
df_city_link_filter = df_city_link[df_city_link['address_city'].str.contains("板橋",na=False)]
df_city_company_all = pd.DataFrame()
df_company_detail_all = pd.DataFrame()
    
pbar1 = tqdm.tqdm(total=len(df_city_link_filter))
for city_name , city_url in zip(df_city_link_filter['address_city'],df_city_link_filter['address_url']) :
        
        df_city_company = get_link_address_sub(basic,city_name,city_url)
        df_city_company_all = pd.concat([df_city_company_all,df_city_company])
        pbar1.update(1)
        time.sleep(0.01)
pbar1.close()

100%|██████████| 1/1 [00:38<00:00, 38.41s/it]


In [118]:
def crawler_all_city_company(basic,city) :  ### city rules is  '台中市|彰化縣|....|'
    
    df_city_link = get_link_address(basic)
    df_city_link_filter = df_city_link[df_city_link['address_city'].str.contains(city,na=False)]
    
    
    df_city_company_all = pd.DataFrame()
    df_company_detail_all = pd.DataFrame()
    
    pbar1 = tqdm.tqdm(total=len(df_city_link_filter))
    for city_name , city_url in zip(df_city_link_filter['address_city'],df_city_link_filter['address_url']) :
        
        df_city_company = get_link_address_sub(basic,city_name,city_url)
        df_city_company_all = pd.concat([df_city_company_all,df_city_company])
        pbar1.update(1)
        time.sleep(0.01)

    pbar1.close()
    
    pbar2 = tqdm.tqdm(total=len(df_city_company_all))
    for address_city,city_company,company_url in zip(df_city_company_all['address_city'],df_city_company_all['city_company']
                           ,df_city_company_all['city_company_url']) :
        
        df_company_detail = get_detail_address_sub(basic,address_city,city_company,company_url)
        df_company_detail_all = pd.concat([df_company_detail_all,df_company_detail])
        pbar2.update(1)
        time.sleep(0.01)

    pbar2.close()
    
    
    return df_city_link_filter , df_city_company_all , df_company_detail_all

In [125]:
t11,t21,t31 = crawler_all_city_company(basic,"板橋")

  0%|          | 4/1000 [00:01<06:31,  2.55it/s]/Users/andychao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

 93%|█████████▎| 929/1000 [25:34<5:26:40, 276.07s/it]

ConnectionError: HTTPSConnectionPool(host='aibee.com.tw', port=443): Max retries exceeded with url: /v/13689261/ (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x122130e90>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [123]:
t31.columns

Index(['address_city', 'category0', 'category1', 'category10', 'category2',
       'category3', 'category4', 'category5', 'category6', 'category7',
       'category8', 'category9', 'city_company', 'industry0', 'industry1',
       'industry2', 'industry3', 'industry4', 'industry5', 'industry6',
       'industry7', '代表人/創辦人/員工: ', '公司名稱2: ', '公司名稱: ', '公司地址: ', '公司狀態: ',
       '實收資本額: ', '撤銷日期: ', '最後核准變更日期: ', '核准設立日期: ', '登記機關名稱: ', '登記種類: ',
       '統一編號: ', '資本額總額: '],
      dtype='object')

In [109]:
t3[(t3['資本額(元): '].apply(lambda x : float(x) >20000 )) | (t3['資本額總額: '].apply(lambda x : float(x) >20000 ))]

NameError: name 't3' is not defined

In [68]:
title , df_city_company = get_link_address_sub(basic,b)

('新北市萬里區總共有 4686 筆商業公司登記資料，以下是 1 - 20 [第 1 頁]。',
                  city_company                  city_company_url
 0          萬金溫泉會舘統一編號26747432  https://aibee.com.tw/v/26747432/
 1    萬美觀光事業股份有限公司統一編號37154574  https://aibee.com.tw/v/37154574/
 2    萬里仙境會館股份有限公司統一編號27286099  https://aibee.com.tw/v/27286099/
 3      萬金休憩股份有限公司統一編號28770367  https://aibee.com.tw/v/28770367/
 4           萬金小吃店統一編號30259510  https://aibee.com.tw/v/30259510/
 ..                        ...                               ...
 495    銓富國際股份有限公司統一編號80664681  https://aibee.com.tw/v/80664681/
 496  亞信檢測科技股份有限公司統一編號80726080  https://aibee.com.tw/v/80726080/
 497          漁進商行統一編號80921696  https://aibee.com.tw/v/80921696/
 498         麗鮮料理店統一編號82109247  https://aibee.com.tw/v/82109247/
 499           詠晟行統一編號82127977  https://aibee.com.tw/v/82127977/
 
 [500 rows x 2 columns])

In [74]:
pd.DataFrame.from_dict(a,orient='index').T

,統一編號:,公司名稱:,公司地址:,資本額總額:,核准設立日期:,最後核准變更日期:,登記機關名稱:,公司狀態:,代表人/創辦人/員工:,登記種類:,...,category1,category2,category3,category4,category5,industry0,industry1,industry2,industry3,category6
0,26747432,萬金溫泉會舘,新北市萬里區萬里加投45之6、45之7、45之8號,240000,2011-11-24,2018-01-29,新北市政府經濟發展局,停業 (停業起迄日期 107年01月09日 至 107年12月31日) - 獨資,林進龍,商業登記,...,一般旅館業,F501060,餐館業,JZ99120,一般浴室業,559099,未分類其他住宿,472913,菸酒零售,營業項目


In [459]:
total_company , address_city_town = get_link_address_sub(basic,list(address_city_link.values())[0])

In [460]:
total_company

'新北市總共有 709125 筆商業公司登記資料，以下是 1 - 20 [第 1 頁]。'

In [461]:
address_city_town

{'永圓國際美容事業股份有限公司台北北新一分公司統一編號24604206': 'https://aibee.com.tw/v/24604206/',
 '新醫藥週刊雜誌社統一編號01462107': 'https://aibee.com.tw/v/01462107/',
 '加美光學股份有限公司統一編號01630738': 'https://aibee.com.tw/v/01630738/',
 '國周企業社統一編號01722194': 'https://aibee.com.tw/v/01722194/',
 '台嶼企業社統一編號01964191': 'https://aibee.com.tw/v/01964191/',
 '南陽實業股份有限公司新北分公司統一編號01978033': 'https://aibee.com.tw/v/01978033/',
 '新永興鑄品社統一編號02099513': 'https://aibee.com.tw/v/02099513/',
 '新豐彩券行統一編號02102947': 'https://aibee.com.tw/v/02102947/',
 '鳴遠商行統一編號02173724': 'https://aibee.com.tw/v/02173724/',
 '新太行統一編號02191700': 'https://aibee.com.tw/v/02191700/',
 '新泰鐘錶隱形眼鏡行統一編號02200378': 'https://aibee.com.tw/v/02200378/',
 '武元興業社統一編號02231105': 'https://aibee.com.tw/v/02231105/',
 '淘爾實業股份有限公司統一編號02266672': 'https://aibee.com.tw/v/02266672/',
 '南北百貨五金行統一編號02272055': 'https://aibee.com.tw/v/02272055/',
 '迎財神到彩券行統一編號02272766': 'https://aibee.com.tw/v/02272766/',
 '縈采百貨行統一編號02275877': 'https://aibee.com.tw/v/02275877/',
 '創威廣告裝潢工程有限公司統一編號02278327

In [472]:
def get_city_data(basic , address_city_town , loading='open') :
    
    temp = []
    
    if loading == 'open' :
        for company_link in tqdm_notebook(address_city_town.values()) :

            new_taipei_list.append(get_detail_address_sub(basic,company_link))
    else :
        for company_link in address_city_town.values() :

            new_taipei_list.append(get_detail_address_sub(basic,company_link))        
            
    df_city_detail = pd.DataFrame.from_dict(temp)
    
    return df_city_detail

In [471]:
new_taipei_list = []
for company_link in tqdm_notebook(address_city_town.values()) :
    
    new_taipei_list.append(get_detail_address_sub(basic,company_link))

df_new_taipei = pd.DataFrame.from_dict(new_taipei_list)

In [430]:
import pandas as pd

In [438]:
b = [a,a]

In [58]:
t3.iloc[:,:15]

,統一編號:,公司名稱:,公司地址:,資本額總額:,核准設立日期:,最後核准變更日期:,公司狀態:,代表人/創辦人/員工:,登記種類:,營業人名稱:,營業地址:,使用統一發票:,設立日期:,資本額(元):,總(本)公司統一編號:
0,53002652,全家便利商店股份有限公司金山金美分公司,新北市金山區中山路250號、252號1樓,0,2010-11-15,2017-02-15,核准設立,葉榮廷,公司登記,全家便利商店股份有限公司金山金美分公司,新北市金山區中山路２５０、２５２號,是,2010-11-15,0,23060248
0,53004531,全家便利商店股份有限公司金山金中分公司,新北市金山區中山路101號、103號1樓,0,2010-12-22,2017-02-15,核准設立,葉榮廷,公司登記,全家便利商店股份有限公司金山金中分公司,新北市金山區中山路１０１、１０３號１樓,是,2010-12-22,0,23060248
0,02240744,勤旺五金行,新北市金山區中山路324號4樓,500000,1994-01-06,1994-01-06,核准設立 - 獨資,吳水朝,商業登記,勤旺五金行,新北市金山區中山路３２４號４樓,是,1994-01-08,500000,NaN
0,02245749,錦德堂香舖,新北市金山區中山路254號1樓,20000,1992-06-11,2014-09-05,核准設立 - 獨資 (核准文號: 1035244372),許宏奎,商業登記,錦德堂香舖,新北市金山區中山路２５４號１樓,否,1992-06-11,20000,NaN
0,02265060,金茂山小吃店,新北市金山區四維街６號１樓,18000,1991-11-16,2006-07-21,核准設立 - 獨資,李金龍,商業登記,金茂山小吃店,新北市金山區四維路６號１樓,否,1991-11-16,18000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,49989299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,陽明山水開發實業股份有限公司金山辦事處,新北市金山區林口３３之３號,是,2005-12-09,0,80690082
0,50655073,金山中山波羅店,新北市金山區中山路199號(1樓),100000,2016-12-20,2017-05-31,歇業 - 獨資 (核准文號: 1068118843),劉忠銘,商業登記,NaN,NaN,NaN,NaN,NaN,NaN
0,50726302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,金山河內越南料理,新北市金山區環金路３９號,否,2016-12-19,500000,NaN
0,70784286,全虹企業股份有限公司金山分公司,新北市金山區和平村中山路１１６號,0,2000-09-01,2005-06-08,廢止,李靜芳,公司登記,NaN,NaN,NaN,NaN,NaN,36057744


In [402]:
key = []
val = []
count = 1
for content in soup.find('div',class_='content').find_all('span') :
    
    if count % 2 != 0 :
        key.append(content.text)
        
    else :
        val.append(content.text)
        
    count += 1

detail = dict(zip(key,val))

In [425]:
detail

{'category': '營業項目', 'industry': '菸酒零售'}

In [380]:
val

['26747432',
 '萬金溫泉會舘',
 '新北市萬里區萬里加投45之6、45之7、45之8號',
 '240000',
 '2011-11-24',
 '2018-01-29',
 '新北市政府經濟發展局',
 '停業 (停業起迄日期 107年01月09日 至 107年12月31日) - 獨資',
 '林進龍',
 '商業登記',
 '萬金溫泉會舘',
 '26747432',
 '新北市萬里區萬里加投４５之６、４５之７、４５之８號',
 '是',
 '2011-11-25',
 '240000',
 ' 新北市 ']

In [413]:
key = []
val = []

for content in soup.find('div',class_='content').find_all('a') :

    if 'category' in content.get('href') :
        key.append('category')
        val.append(content.text)
    elif 'industry' in content.get('href') :
        key.append('industry')
        val.append(content.text)

    else :
        pass


#soup.find('div',class_='content').find_all('a')

In [410]:
for i in soup.find('div',class_='content').find_all('a') :
    if 'category' in i.get('href') :
        print(i.text)
    elif 'industry' in i.get('href') :
        print(i.text)

J901020
一般旅館業
F501060
餐館業
JZ99120
一般浴室業
559099
未分類其他住宿
472913
菸酒零售
營業項目


In [334]:
list(b.values())[0]

'https://aibee.com.tw/address/%E6%96%B0%E5%8C%97%E5%B8%82%E8%90%AC%E9%87%8C%E5%8D%80/'

In [176]:
b = get_link_address(basic)

In [189]:
list(b.values())[0]

'https://aibee.com.tw/address/%E6%96%B0%E5%8C%97%E5%B8%82%E8%90%AC%E9%87%8C%E5%8D%80/'

In [309]:
def get_link_address_sub (basic,address_city_link) :

    
    r = requests.get(address_city_link)
    r.encoding = "UTF-8"
    soup = BeautifulSoup(r.text, 'html.parser')

    total_company = soup.find(class_ = 'content').find('h2').text
    
    basic_page = soup.find(class_ ='NavPg p0').find_all('a')[0].get('href')
    
    try :
        page = (int(soup.find(class_ ='NavPg p0').find_all('a')[-1].get('href')[-4:-1]) -1)  /20
    except :
        page = (int(soup.find(class_ ='NavPg p0').find_all('a')[-1].get('href')[-3:-1]) -1)  /20
    
    
    title_address_city_town = []
    href_address_city_town = []
    for i in range(0,int(page+1)) :
        if i == 0 :
            r = requests.get(basic + basic_page)
            r.encoding = "UTF-8"
            soup = BeautifulSoup(r.text, 'html.parser')
            
            title_address_city_town.extend(list(map(lambda x:x.find('a').get('title'),soup.find_all('li',class_='p0 BbHr'))))
            href_address_city_town.extend(list(map(lambda x:basic + x.find('a').get('href'),soup.find_all('li',class_='p0 BbHr'))))
            
        else :
            r = requests.get(basic + basic_page  +str(i*20 + 1)+'/')
            r.encoding = "UTF-8"
            soup = BeautifulSoup(r.text, 'html.parser')
            title_address_city_town.extend(list(map(lambda x:x.find('a').get('title'),soup.find_all('li',class_='p0 BbHr'))))
            href_address_city_town.extend(list(map(lambda x:basic + x.find('a').get('href'),soup.find_all('li',class_='p0 BbHr'))))
            
    
    address_city_town = dict(zip(title_address_city_town,href_address_city_town))
    
    return total_company , address_city_town

In [306]:
r = requests.get(basic + basic_page  +str(49*20 + 1)+'/')
r.encoding = "UTF-8"
soup = BeautifulSoup(r.text, 'html.parser')

In [316]:
list(d.values())[0]

'https://aibee.com.tw/v/26747432/'

In [307]:
soup

<!DOCTYPE html>

<html itemscope="" itemtype="https://schema.org/WebPage" lang="zh-Hant-TW">
<head prefix="og: http://ogp.me/ns#">
<meta charset="utf-8"/>
<title>新北市萬里區商業公司登記,第50頁:政偉工程有限公司,好玩企劃有限公司 - AiBee台灣公司行號</title>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="yes" name="apple-touch-fullscreen"/>
<meta content="zh_TW" property="og:locale"/>
<meta content="website" property="og:type"/>
<meta content="AiBee台灣公司行號" property="og:site_name"/>
<meta content="新北市萬里區商業公司登記,第50頁:政偉工程有限公司,好玩企劃有限公司 - AiBee台灣公司行號" property="og:title"/>
<meta content="新北市萬里區商業公司登記,第50頁。包含:政偉工程有限公司,好玩企劃有限公司,玉英投資有限公司。總共有 4681 筆商業公司登記資料，以下是 981 - 1000 [第 50 頁]。" property="og:description"/>
<meta content="https://aibee.com.tw/address/%E6%96%B0%E5%8C%97%E5%B8%82%E8%90%AC%E9%87%8C%E5%8D%80/981/" property="og:url"/>
<meta content="https://aibee.com.tw/img-%E6%96%B0%E5%8C%97%E5%B8%82%E8%90%AC%E9%87%8C%E5%8D%80%E5%95%86%E6%A5%AD%E5%85%AC%E5%8F%B8%E7%99%BB%E8%A8%98.png" property="o

In [ ]:
c, d = get_link_address_sub(basic , list(b.values())[0])

In [220]:
r = requests.get('https://aibee.com.tw/address/%E6%96%B0%E5%8C%97%E5%B8%82%E8%90%AC%E9%87%8C%E5%8D%80/')
r.encoding = "UTF-8"
soup = BeautifulSoup(r.text, 'html.parser')

In [311]:
d

{'萬金溫泉會舘統一編號26747432': 'https://aibee.com.tw/v/26747432/',
 '萬美觀光事業股份有限公司統一編號37154574': 'https://aibee.com.tw/v/37154574/',
 '萬里仙境會館股份有限公司統一編號27286099': 'https://aibee.com.tw/v/27286099/',
 '萬金休憩股份有限公司統一編號28770367': 'https://aibee.com.tw/v/28770367/',
 '萬金小吃店統一編號30259510': 'https://aibee.com.tw/v/30259510/',
 '金萬行統一編號33778564': 'https://aibee.com.tw/v/33778564/',
 '萬來企業有限公司統一編號46292000': 'https://aibee.com.tw/v/46292000/',
 '萬投有限公司統一編號67326907': 'https://aibee.com.tw/v/67326907/',
 '萬格企業社統一編號72862033': 'https://aibee.com.tw/v/72862033/',
 '奕萬工程行統一編號82087264': 'https://aibee.com.tw/v/82087264/',
 '萬軍企業社統一編號85172974': 'https://aibee.com.tw/v/85172974/',
 '永鑫機車行統一編號99092353': 'https://aibee.com.tw/v/99092353/',
 '宏全企業社統一編號99495102': 'https://aibee.com.tw/v/99495102/',
 '賢昌企業社統一編號02246402': 'https://aibee.com.tw/v/02246402/',
 '萬里豆漿伯店統一編號09951851': 'https://aibee.com.tw/v/09951851/',
 '萬里鮮水產企業社統一編號10578163': 'https://aibee.com.tw/v/10578163/',
 '宏郁汽車企業社統一編號10589540': 'https://aibee.com.tw/

In [262]:
list(map(lambda x:x.find('a').get('href'),soup.find_all('li',class_='p0 BbHr')))

['/v/26747432/',
 '/v/37154574/',
 '/v/27286099/',
 '/v/28770367/',
 '/v/30259510/',
 '/v/33778564/',
 '/v/46292000/',
 '/v/67326907/',
 '/v/72862033/',
 '/v/82087264/',
 '/v/85172974/',
 '/v/99092353/',
 '/v/99495102/',
 '/v/02246402/',
 '/v/09951851/',
 '/v/10578163/',
 '/v/10589540/',
 '/v/13799730/',
 '/v/14257473/',
 '/v/14657442/']

In [256]:
soup.find('ul',class_='xl0').find_all('li')[-1]

<li><a href="/address/%E6%96%B0%E5%8C%97%E5%B8%82%E8%90%AC%E9%87%8C%E5%8D%80/981/" title="最後頁">&gt;|</a></li>

In [261]:
soup.find_all('li',class_='p0 BbHr')[0]

<li class="p0 BbHr" itemscope="" itemtype="https://schema.org/LocalBusiness">公司商業名稱: <strong itemprop="name"><a href="/v/26747432/" itemprop="url" title="萬金溫泉會舘統一編號26747432">萬金溫泉會舘</a></strong><p>地址: <span itemprop="address">新北市萬里區萬里加投45之6、45之7、45之8號</span> | 統編: <b itemprop="vatID">26747432</b><meta content="https://aibee.com.tw/img-%E8%90%AC%E9%87%91%E6%BA%AB%E6%B3%89%E6%9C%83%E8%88%98.png" itemprop="image"/><meta content="未提供" itemprop="priceRange"/><meta content="未提供" itemprop="telephone"/> | 負責人: 林進龍 | 狀態: 停業 (停業起迄日期 107年01月09日 至 107年12月31日) - 獨資</p></li>

In [204]:
soup.find(class_ ='NavPg p0').find_all('a')

[<a href="/address/%E6%96%B0%E5%8C%97%E5%B8%82%E8%90%AC%E9%87%8C%E5%8D%80/" title="最前頁">|&lt;</a>,
 <a href="/address/%E6%96%B0%E5%8C%97%E5%B8%82%E8%90%AC%E9%87%8C%E5%8D%80/21/" title="第2頁">2</a>,
 <a href="/address/%E6%96%B0%E5%8C%97%E5%B8%82%E8%90%AC%E9%87%8C%E5%8D%80/41/" title="第3頁">3</a>,
 <a href="/address/%E6%96%B0%E5%8C%97%E5%B8%82%E8%90%AC%E9%87%8C%E5%8D%80/61/" title="第4頁">4</a>,
 <a href="/address/%E6%96%B0%E5%8C%97%E5%B8%82%E8%90%AC%E9%87%8C%E5%8D%80/81/" title="第5頁">5</a>,
 <a href="/address/%E6%96%B0%E5%8C%97%E5%B8%82%E8%90%AC%E9%87%8C%E5%8D%80/101/" title="第6頁">6</a>,
 <a href="/address/%E6%96%B0%E5%8C%97%E5%B8%82%E8%90%AC%E9%87%8C%E5%8D%80/121/" title="第7頁">7</a>,
 <a href="/address/%E6%96%B0%E5%8C%97%E5%B8%82%E8%90%AC%E9%87%8C%E5%8D%80/141/" title="第8頁">8</a>,
 <a href="/address/%E6%96%B0%E5%8C%97%E5%B8%82%E8%90%AC%E9%87%8C%E5%8D%80/161/" title="第9頁">9</a>,
 <a href="/address/%E6%96%B0%E5%8C%97%E5%B8%82%E8%90%AC%E9%87%8C%E5%8D%80/181/" title="第10頁">10</a>,
 <a href="/a

In [217]:
soup.find(class_ ='NavPg p0').find_all('a')[0].get('href')

'/address/%E6%96%B0%E5%8C%97%E5%B8%82%E8%90%AC%E9%87%8C%E5%8D%80/'

In [210]:
list(map(lambda x:x.get('href'),soup.find(class_ ='NavPg p0').find_all('a')))[-1][-4:-1]

'981'

In [211]:
981-1 

980

In [212]:
980/20

49.0

In [132]:
soup.find(class_ = 'NavPg p0')

In [90]:
city_link = get_city_link(basic)

In [94]:
city_link['新北市公司行號']

'/address/%E6%96%B0%E5%8C%97%E5%B8%82/'

In [100]:
r = requests.get('https://aibee.com.tw/address/')
r.encoding = "UTF-8"
soup = BeautifulSoup(r.text, 'html.parser')

In [128]:
a = dict()

In [ ]:
a['asdas'] = 

In [63]:
list(map(lambda x :x.get('href'),soup.find_all(class_ = 'p0')[0].find_all('title')))

[]

In [169]:
soup.find(class_ = 'content').find_all('li')[-1].find_all('a')

[<a href="/address/%E9%80%A3%E6%B1%9F%E7%B8%A3%E5%8D%97%E7%AB%BF%E9%84%89/" title="連江縣南竿鄉公司行號">南竿鄉</a>,
 <a href="/address/%E9%80%A3%E6%B1%9F%E7%B8%A3%E5%8C%97%E7%AB%BF%E9%84%89/" title="連江縣北竿鄉公司行號">北竿鄉</a>,
 <a href="/address/%E9%80%A3%E6%B1%9F%E7%B8%A3%E8%8E%92%E5%85%89%E9%84%89/" title="連江縣莒光鄉公司行號">莒光鄉</a>,
 <a href="/address/%E9%80%A3%E6%B1%9F%E7%B8%A3%E6%9D%B1%E5%BC%95%E9%84%89/" title="連江縣東引鄉公司行號">東引鄉</a>]

In [73]:
soup.find(class_ = 'NavPg p0').find_all('a')[-1].get('href')

'/br/cp8.html'

In [26]:
for num in range(len(soup.find_all(class_ = 'p0'))) :
    print(num)

0
1
2
3


In [76]:
soup.find_all(class_ = 'p0')[-1]

<p class="p0"><a href="/address/%E6%96%B0%E5%8C%97%E5%B8%82/" title="新北市公司行號">新北市</a> ■ <a href="/address/%E5%9F%BA%E9%9A%86%E5%B8%82/" title="基隆市公司行號">基隆市</a> ■ <a href="/address/%E8%87%BA%E5%8C%97%E5%B8%82/" title="臺北市公司行號">臺北市</a> ■ <a href="/address/%E6%A1%83%E5%9C%92%E5%B8%82/" title="桃園市公司行號">桃園市</a> ■ <a href="/address/%E6%96%B0%E7%AB%B9%E7%B8%A3/" title="新竹縣公司行號">新竹縣</a> ■ <a href="/address/%E6%96%B0%E7%AB%B9%E5%B8%82/" title="新竹市公司行號">新竹市</a> ■ <a href="/address/%E8%8B%97%E6%A0%97%E7%B8%A3/" title="苗栗縣公司行號">苗栗縣</a> ■ <a href="/address/%E8%87%BA%E4%B8%AD%E5%B8%82/" title="臺中市公司行號">臺中市</a> ■ <a href="/address/%E5%BD%B0%E5%8C%96%E7%B8%A3/" title="彰化縣公司行號">彰化縣</a> ■ <a href="/address/%E5%8D%97%E6%8A%95%E7%B8%A3/" title="南投縣公司行號">南投縣</a> ■ <a href="/address/%E9%9B%B2%E6%9E%97%E7%B8%A3/" title="雲林縣公司行號">雲林縣</a> ■ <a href="/address/%E5%98%89%E7%BE%A9%E7%B8%A3/" title="嘉義縣公司行號">嘉義縣</a> ■ <a href="/address/%E5%98%89%E7%BE%A9%E5%B8%82/" title="嘉義市公司行號">嘉義市</a> ■ <a href="/address/%E8%87